# Lab 9: SMOTE y Perceptrón Simple

In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter

#### Clasificador Euclidiano

In [2]:
# Definir el clasificador de distancia mínima
class ClasificadorDistanciaMinima:
    def ajustar(self, X_entrenamiento, y_entrenamiento):
        # Calcula el centroide para cada clase en el conjunto de entrenamiento
        self.centroides = {}
        clases = set(y_entrenamiento)
        #print(clases)
        X_entrenamiento = X_entrenamiento.apply(pd.to_numeric, errors='coerce')
        for clase in clases:
            muestras_clase = [x for x, y in zip(X_entrenamiento.values, y_entrenamiento) if y == clase]
            #print(X_entrenamiento.dtypes)
            centroide = [sum(caracteristica) / len(muestras_clase) for caracteristica in zip(*muestras_clase)]
            self.centroides[clase] = centroide

    def predecir(self, X_prueba):
        predicciones = []
        X_prueba = X_prueba.apply(pd.to_numeric,errors='coerce')
        for x in X_prueba.values:
            distancias = {clase: sum((xi - ci) ** 2 for xi, ci in zip(x, centroide)) ** 0.5 
                          for clase, centroide in self.centroides.items()}
            predicciones.append(min(distancias, key=distancias.get))
        return predicciones

#### Clasificador 1NN

In [3]:
class clasificador1NN:
    def ajustar(self, X_train, y_train):
        X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)
        y_train = pd.to_numeric(y_train, errors='coerce').fillna(0)
        self.X_train = X_train
        self.y_train = y_train
        
    def predecir(self, X_test):
        y_pred = []
        
        X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)
        
        for _, x in X_test.iterrows():  # Iterar sobre las filas de X_test
            distancias = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            
            idx_vecino = np.argmin(distancias)
            
            y_pred.append(self.y_train.iloc[idx_vecino])
        
        return np.array(y_pred)

#### Perceptron simple

In [ ]:
class Perceptron(object):
    def __init__(self, Learn_Rate=0.5, Iterations=10):
        self.learn_rate = Learn_Rate
        self.Iterations = Iterations
        self.errors = []

    def fit(self, x, y):
        # Inicializar los pesos incluyendo el sesgo (primer peso)
        self.weights = np.zeros(1 + x.shape[1])
        x = x.select_dtypes(include=[np.number])
        x = x.to_numpy()
        for i in range(self.Iterations):
            error = 0
            for xi, target in zip(x, y):
                target_adj = -1 if target == 0 else 1
                update = self.learn_rate * (target_adj - self.predict(xi, convert_output=False))
                self.weights[1:] += update * xi
                self.weights[0] += update
                error += int(update != 0)
            self.errors.append(error)
        return self

    def net_input(self, x):
        return np.dot(x, self.weights[1:]) + self.weights[0]

    def predict(self, x, convert_output=True):
        pred = np.where(self.net_input(x) >= 0.0, 1, -1)
        
        if convert_output:
            return np.where(pred == 1, 2, 0)
        else:
            return pred

#### Hold-Out 

In [5]:
def hold_out_split(data, labels, test_size=0.3):
    combined = data.copy()
    combined['label'] = labels
    
    combined = combined.sample(frac=1, random_state=42).reset_index(drop=True)
    
    split_index = int(len(combined) * (1 - test_size))
    
    train_data = combined.iloc[:split_index, :-1]  # Todas las columnas excepto la última (labels)
    test_data = combined.iloc[split_index:, :-1]
    train_labels = combined.iloc[:split_index, -1]  # Solo la última columna (labels)
    test_labels = combined.iloc[split_index:, -1]
    
    return train_data, test_data, train_labels, test_labels

#### Fold Cross-Validation

In [6]:
def k_fold_split(data, labels, k=5):
    if len(data) != len(labels):
        raise ValueError("El número de muestras en data y labels debe ser el mismo.")

    combined = pd.concat([data, labels], axis=1)

    combined = combined.sample(frac=1, random_state=42).reset_index(drop=True)

    fold_size = len(combined) // k
    remainder = len(combined) % k

    folds = []
    start = 0

    for i in range(k):
        # Calcular el tamaño del fold actual
        current_fold_size = fold_size + (1 if i < remainder else 0)
        end = start + current_fold_size

        # Datos de prueba para el fold actual
        test_data = combined.iloc[start:end, :-1]
        test_labels = combined.iloc[start:end, -1]

        # Datos de entrenamiento: todo excepto el fold actual
        train_data = pd.concat([combined.iloc[:start, :-1], combined.iloc[end:, :-1]], axis=0)
        train_labels = pd.concat([combined.iloc[:start, -1], combined.iloc[end:, -1]], axis=0)

        folds.append((train_data, test_data, train_labels, test_labels))
        start = end

    return folds

#### Matriz de confusion y metricas

In [7]:
def classificationReport(y_true, y_pred):
    # Obtener etiquetas únicas en las predicciones y reales
    labels = sorted(set(y_true) | set(y_pred))
    etiqueta_to_indice = {etiqueta: i for i, etiqueta in enumerate(labels)}
    
    # Calcular la matriz de confusión
    matriz_confusion = np.zeros((len(labels), len(labels)), dtype=int)
    for real, pred in zip(y_true, y_pred):
        i = etiqueta_to_indice[real]
        j = etiqueta_to_indice[pred]
        matriz_confusion[i, j] += 1
    
    # Calcular True Positives, False Positives, y False Negatives globales
    TP = np.diag(matriz_confusion).sum()  # Suma de la diagonal
    FP = matriz_confusion.sum(axis=0) - np.diag(matriz_confusion)
    FN = matriz_confusion.sum(axis=1) - np.diag(matriz_confusion)
    
    # Precisión, Recall, y F1-score generales
    precision_global = TP / (TP + FP.sum()) if (TP + FP.sum()) > 0 else 0
    recall_global = TP / (TP + FN.sum()) if (TP + FN.sum()) > 0 else 0
    f1_score_global = 2 * (precision_global * recall_global) / (precision_global + recall_global) if (precision_global + recall_global) > 0 else 0
    accuracy = TP / matriz_confusion.sum()
    
    # Convertir matriz de confusión a DataFrame para visualización
    matriz_confusion_df = pd.DataFrame(matriz_confusion, index=labels, columns=labels)

    return {
        'precision_global': precision_global,
        'recall_global': recall_global,
        'f1_score_global': f1_score_global,
        'accuracy': accuracy
    }, matriz_confusion_df

#### SMOTE

In [8]:
def smote(X_train,y_train):
    counter = Counter(y_train)
    print('Before', counter)

    # oversampling the train dataset using SMOTE
    smt = SMOTE(random_state=42)
    X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

    counter = Counter(y_train_sm)
    print('After', counter)
    return X_train_sm,y_train_sm   

## Primera parte

In [9]:
glass = pd.read_csv('dataset/glass.csv')
Xglass = glass.drop("Type", axis = 1)
Yglass = glass["Type"]

### Antes SMOKE

In [10]:
# Clasificador con distancia minima
clasificador = ClasificadorDistanciaMinima()
glassX_train, glassX_test, glassY_train, glassY_test = hold_out_split(Xglass, Yglass)

clasificador.ajustar(glassX_train, glassY_train)
predicciones_holdout = clasificador.predecir(glassX_test)
reporte_general, matriz_confusion = classificationReport(glassY_test, predicciones_holdout)

print("**Holdout de distancia minima**")
print("Matriz de Confusión:")
print(matriz_confusion)

print("\nReporte de Clasificación General:")
print(f"Precisión: {reporte_general['precision_global']:.2f}")
print(f"Recall: {reporte_general['recall_global']:.2f}")
print(f"F1-Score: {reporte_general['f1_score_global']:.2f}")
print(f"Exactitud (Accuracy): {reporte_general['accuracy']:.2f}")

#print(predicciones_holdout)

folds = k_fold_split(Xglass, Yglass, k=10)

precision_total = 0
recall_total = 0
f1_score_total = 0
accuracy_total = 0

num_clases = len(set(Yglass))  # Número de clases en el conjunto de datos
matriz_confusion_total = np.zeros((num_clases, num_clases), dtype=int)

for i, (X_train, X_test, y_train, y_test) in enumerate(folds):
    clasificador.ajustar(X_train, y_train)
    predictions = clasificador.predecir(X_test)

    reporte, matriz_confusion = classificationReport(y_test, predictions)
    
    # Acumular las métricas de este fold
    precision_total += reporte['precision_global']
    recall_total += reporte['recall_global']
    f1_score_total += reporte['f1_score_global']
    accuracy_total += reporte['accuracy']
    
    # Asegurar que la matriz de confusión tenga el tamaño adecuado
    if matriz_confusion.shape != matriz_confusion_total.shape:
        # Redimensionar matriz de confusión del fold actual
        matriz_confusion_expanded = np.zeros_like(matriz_confusion_total)
        matriz_confusion_expanded[:matriz_confusion.shape[0], :matriz_confusion.shape[1]] = matriz_confusion
        matriz_confusion = matriz_confusion_expanded

    # Acumular la matriz de confusión
    matriz_confusion_total += matriz_confusion
    
# Calcular promedios de las métricas
num_folds = len(folds)
precision_promedio = precision_total / num_folds
recall_promedio = recall_total / num_folds
f1_score_promedio = f1_score_total / num_folds
accuracy_promedio = accuracy_total / num_folds

# Imprimir matriz de confusión total
print("\n**Fold con distancia minima**")
print("Matriz de Confusión Total:")
print(matriz_confusion_total)

# Imprimir resultados generales promedio
print("Resultados Promedio de Validación Cruzada:")
print(f"Precisión Promedio: {precision_promedio:.2f}")
print(f"Recall Promedio: {recall_promedio:.2f}")
print(f"F1-Score Promedio: {f1_score_promedio:.2f}")
print(f"Exactitud (Accuracy) Promedio: {accuracy_promedio:.2f}")

# Clasificador con 1NN
clasificador = clasificador1NN()
clasificador.ajustar(glassX_train,glassY_train)
predicciones = clasificador.predecir(glassX_test)
reporte_general, matriz_confusion = classificationReport(glassY_test, predicciones)
print("\n**Holdout con 1NN**")
print("Matriz de Confusión:")
print(matriz_confusion)

print("\nReporte de Clasificación General:")
print(f"Precisión: {reporte_general['precision_global']:.2f}")
print(f"Recall: {reporte_general['recall_global']:.2f}")
print(f"F1-Score: {reporte_general['f1_score_global']:.2f}")
print(f"Exactitud (Accuracy): {reporte_general['accuracy']:.2f}")


precision_total = 0
recall_total = 0
f1_score_total = 0
accuracy_total = 0

num_clases = len(set(Yglass))  # Número de clases en el conjunto de datos
matriz_confusion_total = np.zeros((num_clases, num_clases), dtype=int)

for i, (X_train, X_test, y_train, y_test) in enumerate(folds):
    clasificador.ajustar(X_train, y_train)
    predictions = clasificador.predecir(X_test)

    reporte, matriz_confusion = classificationReport(y_test, predictions)
    
    # Acumular las métricas de este fold
    precision_total += reporte['precision_global']
    recall_total += reporte['recall_global']
    f1_score_total += reporte['f1_score_global']
    accuracy_total += reporte['accuracy']
    
    # Asegurar que la matriz de confusión tenga el tamaño adecuado
    if matriz_confusion.shape != matriz_confusion_total.shape:
        # Redimensionar matriz de confusión del fold actual
        matriz_confusion_expanded = np.zeros_like(matriz_confusion_total)
        matriz_confusion_expanded[:matriz_confusion.shape[0], :matriz_confusion.shape[1]] = matriz_confusion
        matriz_confusion = matriz_confusion_expanded

    # Acumular la matriz de confusión
    matriz_confusion_total += matriz_confusion

# Calcular promedios de las métricas
num_folds = len(folds)
precision_promedio = precision_total / num_folds
recall_promedio = recall_total / num_folds
f1_score_promedio = f1_score_total / num_folds
accuracy_promedio = accuracy_total / num_folds

# Imprimir matriz de confusión total
print("\n**Fold con 1NN**")
print("Matriz de Confusión Total:")
print(matriz_confusion_total)

# Imprimir resultados generales promedio
print("Resultados Promedio de Validación Cruzada:")
print(f"Precisión Promedio: {precision_promedio:.2f}")
print(f"Recall Promedio: {recall_promedio:.2f}")
print(f"F1-Score Promedio: {f1_score_promedio:.2f}")
print(f"Exactitud (Accuracy) Promedio: {accuracy_promedio:.2f}")

**Holdout de distancia minima**
Matriz de Confusión:
   1   2  3  5  6   7
1  9   2  8  0  0   0
2  1  10  5  6  0   0
3  1   0  3  0  0   0
5  0   0  0  4  0   0
6  0   1  0  0  2   0
7  0   0  1  2  0  10

Reporte de Clasificación General:
Precisión: 0.58
Recall: 0.58
F1-Score: 0.58
Exactitud (Accuracy): 0.58

**Fold con distancia minima**
Matriz de Confusión Total:
    1   2   3   5   6   7
1  40   5  25   0   0   0
2  37  11  17  10   1   0
3  11   0   6   0   0   0
5   0   0   0  12   2   0
6   0   2   1   0  13   1
7   0   0   3   1   1  15
Resultados Promedio de Validación Cruzada:
Precisión Promedio: 0.45
Recall Promedio: 0.45
F1-Score Promedio: 0.45
Exactitud (Accuracy) Promedio: 0.45

**Holdout con 1NN**
Matriz de Confusión:
    1   2  3  5  6  7
1  15   3  1  0  0  0
2   5  16  0  1  0  0
3   3   0  1  0  0  0
5   0   0  0  4  0  0
6   0   1  0  0  2  0
7   1   1  1  0  1  9

Reporte de Clasificación General:
Precisión: 0.72
Recall: 0.72
F1-Score: 0.72
Exactitud (Accuracy): 

### Despues de SMOKE

In [11]:
Xglass,Yglass = smote(Xglass,Yglass)

# Clasificador con distancia minima
clasificador = ClasificadorDistanciaMinima()
glassX_train, glassX_test, glassY_train, glassY_test = hold_out_split(Xglass, Yglass)

clasificador.ajustar(glassX_train, glassY_train)
predicciones_holdout = clasificador.predecir(glassX_test)
reporte_general, matriz_confusion = classificationReport(glassY_test, predicciones_holdout)

print("**Holdout de distancia minima**")
print("Matriz de Confusión:")
print(matriz_confusion)

print("\nReporte de Clasificación General:")
print(f"Precisión: {reporte_general['precision_global']:.2f}")
print(f"Recall: {reporte_general['recall_global']:.2f}")
print(f"F1-Score: {reporte_general['f1_score_global']:.2f}")
print(f"Exactitud (Accuracy): {reporte_general['accuracy']:.2f}")

#print(predicciones_holdout)

folds = k_fold_split(Xglass, Yglass, k=10)

precision_total = 0
recall_total = 0
f1_score_total = 0
accuracy_total = 0

num_clases = len(set(Yglass))  # Número de clases en el conjunto de datos
matriz_confusion_total = np.zeros((num_clases, num_clases), dtype=int)

for i, (X_train, X_test, y_train, y_test) in enumerate(folds):
    clasificador.ajustar(X_train, y_train)
    predictions = clasificador.predecir(X_test)

    reporte, matriz_confusion = classificationReport(y_test, predictions)
    
    # Acumular las métricas de este fold
    precision_total += reporte['precision_global']
    recall_total += reporte['recall_global']
    f1_score_total += reporte['f1_score_global']
    accuracy_total += reporte['accuracy']
    
    # Asegurar que la matriz de confusión tenga el tamaño adecuado
    if matriz_confusion.shape != matriz_confusion_total.shape:
        # Redimensionar matriz de confusión del fold actual
        matriz_confusion_expanded = np.zeros_like(matriz_confusion_total)
        matriz_confusion_expanded[:matriz_confusion.shape[0], :matriz_confusion.shape[1]] = matriz_confusion
        matriz_confusion = matriz_confusion_expanded

    # Acumular la matriz de confusión
    matriz_confusion_total += matriz_confusion
    
# Calcular promedios de las métricas
num_folds = len(folds)
precision_promedio = precision_total / num_folds
recall_promedio = recall_total / num_folds
f1_score_promedio = f1_score_total / num_folds
accuracy_promedio = accuracy_total / num_folds

# Imprimir matriz de confusión total
print("\n**Fold con distancia minima**")
print("Matriz de Confusión Total:")
print(matriz_confusion_total)

# Imprimir resultados generales promedio
print("Resultados Promedio de Validación Cruzada:")
print(f"Precisión Promedio: {precision_promedio:.2f}")
print(f"Recall Promedio: {recall_promedio:.2f}")
print(f"F1-Score Promedio: {f1_score_promedio:.2f}")
print(f"Exactitud (Accuracy) Promedio: {accuracy_promedio:.2f}")

# Clasificador con 1NN
clasificador = clasificador1NN()
clasificador.ajustar(glassX_train,glassY_train)
predicciones = clasificador.predecir(glassX_test)
reporte_general, matriz_confusion = classificationReport(glassY_test, predicciones)
print("\n**Holdout con 1NN**")
print("Matriz de Confusión:")
print(matriz_confusion)

print("\nReporte de Clasificación General:")
print(f"Precisión: {reporte_general['precision_global']:.2f}")
print(f"Recall: {reporte_general['recall_global']:.2f}")
print(f"F1-Score: {reporte_general['f1_score_global']:.2f}")
print(f"Exactitud (Accuracy): {reporte_general['accuracy']:.2f}")


precision_total = 0
recall_total = 0
f1_score_total = 0
accuracy_total = 0

num_clases = len(set(Yglass))  # Número de clases en el conjunto de datos
matriz_confusion_total = np.zeros((num_clases, num_clases), dtype=int)

for i, (X_train, X_test, y_train, y_test) in enumerate(folds):
    clasificador.ajustar(X_train, y_train)
    predictions = clasificador.predecir(X_test)

    reporte, matriz_confusion = classificationReport(y_test, predictions)
    
    # Acumular las métricas de este fold
    precision_total += reporte['precision_global']
    recall_total += reporte['recall_global']
    f1_score_total += reporte['f1_score_global']
    accuracy_total += reporte['accuracy']
    
    # Asegurar que la matriz de confusión tenga el tamaño adecuado
    if matriz_confusion.shape != matriz_confusion_total.shape:
        # Redimensionar matriz de confusión del fold actual
        matriz_confusion_expanded = np.zeros_like(matriz_confusion_total)
        matriz_confusion_expanded[:matriz_confusion.shape[0], :matriz_confusion.shape[1]] = matriz_confusion
        matriz_confusion = matriz_confusion_expanded

    # Acumular la matriz de confusión
    matriz_confusion_total += matriz_confusion

# Calcular promedios de las métricas
num_folds = len(folds)
precision_promedio = precision_total / num_folds
recall_promedio = recall_total / num_folds
f1_score_promedio = f1_score_total / num_folds
accuracy_promedio = accuracy_total / num_folds

# Imprimir matriz de confusión total
print("\n**Fold con 1NN**")
print("Matriz de Confusión Total:")
print(matriz_confusion_total)

# Imprimir resultados generales promedio
print("Resultados Promedio de Validación Cruzada:")
print(f"Precisión Promedio: {precision_promedio:.2f}")
print(f"Recall Promedio: {recall_promedio:.2f}")
print(f"F1-Score Promedio: {f1_score_promedio:.2f}")
print(f"Exactitud (Accuracy) Promedio: {accuracy_promedio:.2f}")

Before Counter({2: 76, 1: 70, 7: 29, 3: 17, 5: 13, 6: 9})
After Counter({1: 76, 2: 76, 3: 76, 5: 76, 6: 76, 7: 76})
**Holdout de distancia minima**
Matriz de Confusión:
    1  2   3   5   6   7
1  10  6  11   0   0   0
2   6  5   3   3   0   0
3   4  0  22   0   0   0
5   0  1   1  21   0   0
6   0  0   0   0  17   1
7   0  0   2   1   3  20

Reporte de Clasificación General:
Precisión: 0.69
Recall: 0.69
F1-Score: 0.69
Exactitud (Accuracy): 0.69

**Fold con distancia minima**
Matriz de Confusión Total:
    1   2   3   5   6   7
1  41   7  28   0   0   0
2  27  12  26  10   1   0
3  20   0  56   0   0   0
5   0   1   3  65   0   7
6   0   0   0   3  69   4
7   0   0   5   2   8  61
Resultados Promedio de Validación Cruzada:
Precisión Promedio: 0.67
Recall Promedio: 0.67
F1-Score Promedio: 0.67
Exactitud (Accuracy) Promedio: 0.67

**Holdout con 1NN**
Matriz de Confusión:
    1   2   3   5   6   7
1  19   4   4   0   0   0
2   4  12   0   1   0   0
3   3   0  23   0   0   0
5   0   0   0 

## Segunda parte

In [12]:
iris = pd.read_csv('iris_clean.csv')
irisClass = iris[iris['class_encoded'].isin([0,2])]
Xiris = irisClass.drop(columns=['class','class_encoded'])
Yiris = irisClass['class_encoded']

In [14]:
perceptron = Perceptron(Learn_Rate=0.01, Iterations=100)
X_train, X_test, y_train, y_test = hold_out_split(Xiris, Yiris)
X_train = X_train.astype(float)
y_train = y_train.astype(float)

# Entrenar el modelo
perceptron.fit(X_train, y_train)

# Realizar predicciones
predicciones = perceptron.predict(X_test)

reporte_general, matriz_confusion = classificationReport(y_test, predicciones)
print("\n**Holdout con Perceptron**")
print("Matriz de Confusión:")
print(matriz_confusion)

print("\nReporte de Clasificación General:")
print(f"Precisión: {reporte_general['precision_global']:.2f}")
print(f"Recall: {reporte_general['recall_global']:.2f}")
print(f"F1-Score: {reporte_general['f1_score_global']:.2f}")
print(f"Exactitud (Accuracy): {reporte_general['accuracy']:.2f}")


**Holdout con Perceptron**
Matriz de Confusión:
    0   2
0  11   0
2   0  19

Reporte de Clasificación General:
Precisión: 1.00
Recall: 1.00
F1-Score: 1.00
Exactitud (Accuracy): 1.00
